In [1]:
#!fsharp
#!value --from-file input.txt --name input

In [1]:
#!fsharp
#!share input --from value

In [1]:
#!fsharp
let inline (.+) (x1, y1) (x2, y2) = (x1 + x2, y1 + y2)
let inline (.*) m (x,y) = (m * x, m * y)

[<Measure>] type degree;

type Instruction =
  | Absolute of int * int
  | Rotation of int<degree>
  | Forward of int

let (|CardinalDirection|_|) = function
  | 'E' -> (1,0) |> Some
  | 'N' -> (0,1) |> Some
  | 'W' -> (-1,0) |> Some
  | 'S' -> (0,-1) |> Some
  | _ -> None

let parseInstruction (s:string) = 
  match (s.[0], int s.[1..]) with
  | CardinalDirection d, i -> i .* d |> Absolute
  | 'L', i -> i * 1<degree> |> Rotation
  | 'R', i -> i * -1<degree> |> Rotation
  | 'F', i -> Forward i
  | _ -> failwith s

let parse (s:string) =
  s.Trim().Split()
  |> Seq.map parseInstruction
  |> Seq.toList

parse """F10
N3
F7
R90
F11""" |> sprintf "%A"

[Forward 10; Absolute (0, 3); Forward 7; Rotation -90; Forward 11]

In [1]:
#!fsharp
let rotate r = 
  let inline (%!) x y = (x % y + y) % y // Actual modulo, not remainder
  let (a,b,c,d) =
    match r %! 360<degree> with
    | 0<degree> -> (1,0,0,1)
    | 90<degree> -> (0,-1,1,0)
    | 180<degree> -> (-1,0,0,-1)
    | 270<degree> -> (0,1,-1,0)
    | _ -> failwith "Only supports right-angle rotations"
  fun (x,y) -> (a * x + b * y, c * x + d * y)

let go (pos, heading) = function
  | Absolute (x,y) -> pos .+ (x,y), heading
  | Rotation r -> pos, rotate r heading
  | Forward m -> pos .+ m .* heading, heading

let runAll f init = Seq.fold f init >> fst

let manhattan (x,y) = abs x + abs y

let instructions = parse input

instructions 
|> runAll go ((0,0), (1,0))
|> manhattan

1148

In [1]:
#!fsharp
let go2 (pos, heading) = function
  | Absolute (x,y) -> pos, heading .+ (x,y)
  | x -> go (pos, heading) x

instructions
|> runAll go2 ((0,0), (10,1))
|> manhattan

52203